In [1]:
from nnsight import LanguageModel
from tqdm import tqdm
from transformers import AutoTokenizer
import torch as t

/private/groups/ioannidislab/cthoma13/gptoss_bullshit_detector/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
lm = LanguageModel(
    "openai/gpt-oss-20b",
    device_map="auto",
    torch_dtype=t.bfloat16,
)

In [3]:
prompts = [
    "hij svkasdjivaijdsvaijsd ssd asdv asd vas dv a", 
    "how sfgsfa ", 
    "are sjpfbwjp fbjpsdfbj opasdj bajsbapdsobpjv asp ojdbaojsdbv apjosdpo vas pdob opjasjo", 
    "you a sdpjo pasdjo"
]

In [4]:
messages = [
    [
    {"role": "system", "content": "Do sth avj dsjiavsja"},
    {"role": "user", "content": user_prompt},
    ]
    for user_prompt in prompts
]

In [5]:
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    padding=True,
    return_tensors="pt",
    return_dict=True,
)

In [6]:
stop_token_id = 199999

In [7]:
prompt_lens = [(input_ids != stop_token_id).sum() for input_ids in inputs["input_ids"]]

In [8]:
prompt_lens

[tensor(102), tensor(89), tensor(121), tensor(91)]

In [9]:
steering_vec = t.ones((1, 1, lm.config.hidden_size), dtype=t.bfloat16).to(lm.device)

In [10]:
with lm.generate(inputs, max_new_tokens=4096):
    with lm.model.layers.all():
        h = lm.model.layers[0].output[0]                                 # <-- select hidden-state slot
        steered = h - 1.0 * steering_vec                                        # <-- no in-place; broadcasting ok
        lm.model.layers[0].output[0] = steered
    generated_ids = lm.generator.output.save()

MXFP4 quantization requires triton >= 3.4.0 and triton_kernels installed, we will default to dequantizing the model to bf16
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.41it/s]
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


NNsightError: Cannot copy out of meta tensor; no data!

In [ ]:
generated_ids.shape

torch.Size([4, 1, 2880])